In [1]:
"""
This notebook is for making full classification plots.
"""

'\nThis notebook is for making full classification plots.\n'

In [29]:
import numpy as np

import glob
import os
import scipy.io as sio
import imageio


In [2]:
import hsi_data

/scratch0/ilya/locDoc/miniconda2/envs/venvtf/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import ipympl

In [4]:
pavia_class_map = [[192, 192, 192], [41, 253, 47], [45, 255, 253], [14, 126, 18], [250, 40, 251], [162, 81, 43], [126, 11, 123], [247, 9, 27], [254, 253, 56] ]
botswana_class_map = [[77, 104, 255], [118, 149, 226], [237, 243, 57], [171, 153, 58], [236, 185, 46], [218, 2, 0], [124, 47, 225], [231, 254, 226], [105, 2, 0], [222, 217, 251], [248, 245, 139], [234, 179, 193], [175, 102, 41], [245, 245, 245]]
ksc_class_map = [[136, 203, 71], [218, 0, 249], [192, 114, 27], [151, 34, 2], [31, 54, 5], [64, 0, 0], [255, 255, 255], [139, 132, 131], [255, 254, 174], [240, 196, 86], [122, 199, 251], [38, 65, 119], [1, 2, 248]]
ip_class_map = [[98, 203, 178], [252, 40, 247], [212, 113, 29], [175, 31, 16], [22, 133, 250], [68, 1, 2], [254, 254, 254], [143, 133, 132], [255, 253, 175], [254, 196, 91], [66, 199, 45], [18, 90, 124], [7, 36, 249], [40, 220, 240], [160, 93, 150], [120, 40, 230]]

In [5]:
def apply_colormap(img, class_map):
    """
    class_map: list< list< int >( 3 ) >
        maps classes to colors
    """
    h, w = img.shape
    new_img = np.zeros((h*w,3))
    for i, p in enumerate(img.flatten()):
        if p == -1:
            new_img[i] = [0,0,0]
        else:
            new_img[i] = class_map[p]
    new_img = new_img / 255.
    return new_img.reshape((h,w,3))
        

In [31]:
def load_npz_results_from_dirs(res_files):
    """
    Args:
      res_files: list<str>: list of full paths to results.npz files
    Returns:
      dir_results: dict< str, float >: maps mask name to accuracy
    """
    dir_results = {}
    for npz_name in res_files:
        results = np.load(npz_name, allow_pickle=True)['results'].item()
        k = results.keys()[0]
        for k, v in results.iteritems():
            if k in dir_results:
                dir_results[k] = max(dir_results[k], v)
            else:
                dir_results[k] = v
    return dir_results
    

In [358]:
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'PaviaU_distributed_trainval_p*/*/results.npz'))
results = load_npz_results_from_dirs(res_files)

In [369]:
npz_name = '/scratch0/ilya/locDoc/pyfst/models/pu_tang_svm/SVM_results_1051942010.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()

In [370]:
# pick best performing mask, then run the prediction with it
best_acc = 0
best = None
for k,v in [(k,v) for k,v in results.items() if 's10' in k]:
    if v > best_acc:
        best_acc = v
        best = k
print(best)

/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/PaviaU_distributed_trainval_s10_7_759773.mat


In [338]:
# npf = np.load('/scratch0/ilya/locDoc/pyfst/june_models/ksc_sss_raw_svm/KSC_gt_SVM.npz')
npf = np.load('/scratch1/ilya/locDoc/pyfst/june_models/ip_sss_fst/IP_strictsinglesite_trainval_s05_8_351788_pred.npz')
img_toshow = apply_colormap(npf['pred_image'], ip_class_map)

In [339]:
fig = plt.figure()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
ax.imshow(img_toshow)

FigureCanvasNbAgg()

In [334]:
imageio.imsave('/scratch0/ilya/locDownloads/temp/tgrs_grids/Botswana_dist_wst.png', img_toshow)

In [307]:
plt.close('all')